In [1]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [2]:
import argparse
import os
from solver import Solver
import glob
from torchvision import transforms, utils
import torchvision.transforms as standard_transforms
from torch.utils.data import Dataset, DataLoader
from own_dataloader import Rescale
from own_dataloader import RescaleT
from own_dataloader import RandomCrop
from own_dataloader import CenterCrop
from own_dataloader import ToTensor
from own_dataloader import ToTensorLab
from own_dataloader import SalObjDataset

In [3]:
#data_dir =   'C:/Users/tip/Documents/GitHub/Saliency_Dataset/dataset_test/DUTOMRON/'
#data_dir =   'C:/Users/tip/Documents/GitHub/Saliency_Dataset/dataset_test/DUTS-TE/'
#data_dir =   'C:/Users/tip/Documents/GitHub/Saliency_Dataset/dataset_test/ECSSD/'
#data_dir =   'C:/Users/tip/Documents/GitHub/Saliency_Dataset/dataset_test/HKU-IS/'
data_dir =   'C:/Users/tip/Documents/GitHub/Saliency_Dataset/dataset_test/PASCALS/'
#data_dir =   'C:/Users/tip/Documents/GitHub/Saliency_Dataset/dataset_test/SOD/'
#tra_image_dir = 'DUTS-TR/DUTS-TR-Image/'
#tra_label_dir = 'DUTS-TR/DUTS-TR-Mask/'
test_image_dir = 'Imgs/'
test_label_dir = 'gt/'
enableInpaintAug = False
batch_size_train=32
batch_size_val=32

image_ext = '.jpg'
label_ext = '.png'

vgg_path = 'D:/nonat project/Experiment 1/weights/vgg16_feat.pth'
trained_model='D:/nonat project/Experiment 1/weights/Experiment1.pth'

#test_folder='D:/nonat project/Experiment 1/weights/test/DUTOMRON TEST'
#test_folder='D:/nonat project/Experiment 1/weights/test/DUTS TEST'
#test_folder='D:/nonat project/Experiment 1/weights/test/ECCSD TEST'
#test_folder='D:/nonat project/Experiment 1/weights/test/HKU-IS TEST'
test_folder='D:/nonat project/Experiment 1/weights/test/PASCAL TEST'
#test_folder='D:/nonat project/Experiment 1/weights/test/SOD TEST'

#output_path='D:/nonat project/Experiment 1/Experiment 1 Predictions/DUTS OMRON Saliency Map Prediction/'
#output_path='D:/nonat project/Experiment 1/Experiment 1 Predictions/DUTS Saliency Map Prediction/'
#output_path='D:/nonat project/Experiment 1/Experiment 1 Predictions/ECCSD Saliency Map Prediction/'
#output_path='D:/nonat project/Experiment 1/Experiment 1 Predictions/HKU-IS Saliency Map Prediction/'
output_path='D:/nonat project/Experiment 1/Experiment 1 Predictions/PASCAL Saliency Map Prediction/'
#output_path='D:/nonat project/Experiment 1/Experiment 1 Predictions/SOD Saliency Map Prediction/'

In [4]:
def main(config):

    if config.mode == 'train':
        tra_img_name_list = glob.glob(data_dir + tra_image_dir + '*' + image_ext)
        print("data_dir + tra_image_dir + '*' + image_ext: ", data_dir + tra_image_dir + '*' + image_ext)

        tra_lbl_name_list = []
        for img_path in tra_img_name_list:
        	img_name = img_path.split("\\")[-1]
        	aaa = img_name.split(".")
        	bbb = aaa[0:-1]
        	imidx = bbb[0]
        	for i in range(1,len(bbb)):
        		imidx = imidx + "." + bbb[i]
        	tra_lbl_name_list.append(data_dir + tra_label_dir + imidx + label_ext)

        print("---")
        print("train images: ", len(tra_img_name_list))
        print("train labels: ", len(tra_lbl_name_list))
        print("---")

        train_num = len(tra_img_name_list)
        salobj_dataset = SalObjDataset(
            img_name_list=tra_img_name_list,
            lbl_name_list=tra_lbl_name_list,
            transform=transforms.Compose([
                RescaleT(256),
                RandomCrop(224),
                ToTensorLab(flag=0)]),
        		category="train",
        		enableInpaintAug=enableInpaintAug)

    test_img_name_list = glob.glob(data_dir + test_image_dir + '*' + image_ext)
    print("data_dir + test_image_dir + '*' + image_ext: ", data_dir + test_image_dir + '*' + image_ext)
    test_lbl_name_list = []
    for img_path in test_img_name_list:
    	img_name = img_path.split("\\")[-1]
    	aaa = img_name.split(".")
    	bbb = aaa[0:-1]
    	imidx = bbb[0]
    	for i in range(1,len(bbb)):
    		imidx = imidx + "." + bbb[i]
    	test_lbl_name_list.append(data_dir + test_label_dir + imidx + label_ext)

    print("---")
    print("test images: ", len(test_img_name_list))
    print("test labels: ", len(test_lbl_name_list))
    print("---")

    test_num = len(test_img_name_list)
    salobj_dataset_test = SalObjDataset(
        img_name_list=test_img_name_list,
        lbl_name_list=test_lbl_name_list,
        transform=transforms.Compose([
            RescaleT(256),
            RandomCrop(224),
            ToTensorLab(flag=0)]),
    		category="test",
    		enableInpaintAug=enableInpaintAug)
    if config.mode == 'train':
        train_loader = DataLoader(salobj_dataset, batch_size=config.batch_size, shuffle=True, num_workers=1)
        if config.val:
        	val_loader = DataLoader(salobj_dataset_test, batch_size=config.batch_size_val, shuffle=True, num_workers=1)
        run = 0
        while os.path.exists("%s/run-%d" % (config.save_fold, run)): run += 1
        os.mkdir("%s/run-%d" % (config.save_fold, run))
        os.mkdir("%s/run-%d/logs" % (config.save_fold, run))
        # os.mkdir("%s/run-%d/images" % (config.save_fold, run))
        os.mkdir("%s/run-%d/models" % (config.save_fold, run))
        config.save_fold = "%s/run-%d" % (config.save_fold, run)
        if config.val:
            train = Solver(train_loader, val_loader, None, config)
        else:
            train = Solver(train_loader, None, None, config)
        train.train(100)
    elif config.mode == 'test':
        test_loader = DataLoader(salobj_dataset_test, batch_size=batch_size_val, shuffle=True, num_workers=1)
        if not os.path.exists(config.test_fold): os.mkdir(config.test_fold)
        test = Solver(None, None, test_loader, config)
        test.test(100,output_path, use_crf=config.use_crf)
    else:
        raise IOError("illegal input!!!")

In [5]:
if __name__ == '__main__':
    data_root = os.path.join(os.path.expanduser('~'), 'data')

    parser = argparse.ArgumentParser()

    # Hyper-parameters
    parser.add_argument('--n_color', type=int, default=3)
    parser.add_argument('--img_size', type=int, default=256)  # 256
    parser.add_argument('--lr', type=float, default=1e-6)
    parser.add_argument('--clip_gradient', type=float, default=1.0)
    parser.add_argument('--cuda', type=bool, default=False)

    # Training settings
    parser.add_argument('--vgg', type=str, default=vgg_path)
    parser.add_argument('--epoch', type=int, default=500)
    parser.add_argument('--val', type=bool, default=True)

    parser.add_argument('--num_thread', type=int, default=4)
    parser.add_argument('--load', type=str, default='')
    parser.add_argument('--save_fold', type=str, default='./results')
    parser.add_argument('--epoch_val', type=int, default=10)
    parser.add_argument('--batch_size', type=int, default=batch_size_train)
    parser.add_argument('--batch_size_val', type=int, default=batch_size_val)
    parser.add_argument('--epoch_save', type=int, default=10)
    parser.add_argument('--epoch_show', type=int, default=1)
    parser.add_argument('--pre_trained', type=str, default=None)

    # Testing settings
    parser.add_argument('--model', type=str, default=trained_model)
    parser.add_argument('--test_fold', type=str, default='./results/test')
    parser.add_argument('--use_crf', type=bool, default=False)

    # Misc
    parser.add_argument('--mode', type=str, default='train', choices=['train', 'test'])
    parser.add_argument('--visdom', type=bool, default=False)

    import easydict
    
    config =  easydict.EasyDict({
    
   "n_color": 3,
    "img_size":256,
    "lr":1e-6,
    "clip_gradient":1.0,
    "cuda":True,

    "vgg":vgg_path,
    "epoch":500,
    "val":True,
        
    "num_thread":4,
    "load":'',
    "save_fold":'./results',
    "epoch_val":10,
    "batch_size":batch_size_train,
    "batch_size_val":batch_size_val,
    "epoch_save":10,
    "epoch_show":1,
    "pre_trained":None,

    "model":trained_model,
    "test_fold":'./results/test',
    "use_crf":False,

    "mode":'test',
    "visdom":False,
})
    if not os.path.exists(config.save_fold): os.mkdir(config.save_fold)
    main(config)
    
    

data_dir + test_image_dir + '*' + image_ext:  C:/Users/tip/Documents/GitHub/Saliency_Dataset/dataset_test/PASCALS/Imgs/*.jpg
---
test images:  850
test labels:  850
---
32
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
[0] mae: 0.0938
32
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
[1] mae: 0.0736
32
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
[2] mae: 0.0789
32
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
[3] mae: 0.1053
32
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
[4] mae: 0.0846
32
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
[5] mae: 0.0750
32
192
193
194
195
196
197
198
199
200
201
202
203
204
205
